In [1]:
from  confluent_kafka import Producer
from config import *
import csv  
import time
import json  

In [2]:
import sys
import importlib
importlib.reload(sys.modules['config'])

<module 'config' from 'e:\\TrungPhanADVN\\Code\\Project_DE\\Final_project_zoomcamp\\streaming_ingest_data\\config.py'>

In [13]:
def read_csv(file_path):
    count = 0
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            yield row
            

In [9]:
read_csv('../data/202201-divvy-tripdata.csv')

<generator object read_csv at 0x000001AC49E33350>

In [14]:
def delivery_report(err, msg):
    if err is not None:
        print(f'Message delivery failed: {err}')
    else:
        print(f'Message delivered to topic: {msg.topic()}, partition: {msg.partition()}, offset: {msg.offset()}')

In [15]:
def produce_data(bootstrap_servers, topic, file_path):
    producer = Producer({'bootstrap.servers':bootstrap_servers})
    try:
        for row in read_csv(file_path):
            json_row = json.dumps(row)
            producer.produce(topic, value=json_row, callback=delivery_report)
            producer.flush()
            break
    except KeyboardInterrupt:
        pass 
    finally:
        producer.flush()
    


In [17]:
produce_data(BOOTSTRAP_SERVERS,TOPIC, '../data/202201-divvy-tripdata.csv')

Message delivered to topic: cycle_data, partition: 0, offset: 1302


In [61]:
BOOTSTRAP_SERVERS

'localhost:9092'

In [44]:
from confluent_kafka import Consumer, KafkaError

In [40]:
import sys
import importlib
import sys, os
sys.path.append(os.path.abspath("./"))
importlib.reload(sys.modules['config'])

<module 'config' from 'e:\\TrungPhanADVN\\Code\\Project_DE\\Final_project_zoomcamp\\streaming_ingest_data\\config.py'>

In [42]:
conf ={'bootstrap.servers':BOOTSTRAP_SERVERS,
       'group.id': 'cycle_data',
       'auto.offset.reset': 'earliest'
       }

In [43]:
consumer = Consumer(conf)

In [51]:
def consume_data(bootstrap_servers, topic):
    conf = {'bootstrap.servers': bootstrap_servers, 'group.id': 'ashraf-supply-chain', 'auto.offset.reset': 'earliest'}
    consumer = Consumer(conf)
    consumer.subscribe([topic])

    try:
        while True:
            msg = consumer.poll(timeout=1.0)
            if msg is None:
                continue
            if msg.error():
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    # End of partition
                    continue
                else:
                    print(msg.error())
                    break
            else:
                # Message value and key are raw bytes -- decode if necessary!
                # e.g., convert to utf-8
                print('Received message: {}'.format(msg.value().decode('utf-8')))
                print('Received message: {}'.format(msg.value()))
                print('Received message: {}'.format(json.loads(msg.value())))

            break
    except KeyboardInterrupt:
        pass

    finally:
        consumer.close()

In [53]:
consume_data(BOOTSTRAP_SERVERS, TOPIC)

Received message: {"ride_id": "C2F7DD78E82EC875", "rideable_type": "electric_bike", "started_at": "2022-01-13 11:59:47", "ended_at": "2022-01-13 12:02:44", "start_station_name": "Glenwood Ave & Touhy Ave", "start_station_id": "525", "end_station_name": "Clark St & Touhy Ave", "end_station_id": "RP-007", "start_lat": "42.0128005", "start_lng": "-87.665906", "end_lat": "42.01256011541", "end_lng": "-87.6743671152", "member_casual": "casual"}
Received message: b'{"ride_id": "C2F7DD78E82EC875", "rideable_type": "electric_bike", "started_at": "2022-01-13 11:59:47", "ended_at": "2022-01-13 12:02:44", "start_station_name": "Glenwood Ave & Touhy Ave", "start_station_id": "525", "end_station_name": "Clark St & Touhy Ave", "end_station_id": "RP-007", "start_lat": "42.0128005", "start_lng": "-87.665906", "end_lat": "42.01256011541", "end_lng": "-87.6743671152", "member_casual": "casual"}'
Received message: {'ride_id': 'C2F7DD78E82EC875', 'rideable_type': 'electric_bike', 'started_at': '2022-01-13

In [18]:
from kafka import KafkaAdminClient


admin_client = KafkaAdminClient(bootstrap_servers=[BOOTSTRAP_SERVERS])
admin_client.delete_topics(topics=[TOPIC, 'cycle_data_2'])

DeleteTopicsResponse_v3(throttle_time_ms=0, topic_error_codes=[(topic='cycle_data', error_code=0), (topic='cycle_data_2', error_code=0)])